In [1]:
from secret_key import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key

#print(openapi_key)
print(os.environ['OPENAI_API_KEY'])

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain
import json


model = OllamaLLM(model="llama3.2:1b", temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template = "I want to open an restaurent for {cuisine} food. Suggest a fancy name. Return only one name without any other text"
)

prompt_template_name.format(cuisine="Mexican")

name_chain = LLMChain(llm=model, prompt=prompt_template_name)

o = name_chain.invoke("Gujarati")

outjson = json.loads( json.dumps(o))

print( "generated name: " + outjson['text'] ) 

/var/folders/py/tj_ky_qj1tn2jgg5gxfj2t5r0000gq/T/ipykernel_87043/2875111840.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(llm=model, prompt=prompt_template_name)


generated name: "Khodiyar's"


In [12]:
prompt_template_items = PromptTemplate(
    input_variables= ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list"
)

food_items_chain = LLMChain(llm=model, prompt=prompt_template_items)


In [13]:
from langchain.chains.sequential import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain] )

response = chain.run("Indian")
print( response )


/var/folders/py/tj_ky_qj1tn2jgg5gxfj2t5r0000gq/T/ipykernel_87043/2171301943.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run("Indian")


Here are some menu item suggestions for Tandoori Nights:

Grilled chicken tikka, naan bread with garlic butter, palak paneer, saag aloo, tandoori vegetable skewers, basmati rice pilaf, chaat items like papdi chaat and raita, fruit and yogurt salad, gajar ka halwa, kulfi, and kachori.


In [14]:
prompt_template_name2 = PromptTemplate(
    input_variables= ['cuisine'],
    template = "I want to open an restaurent for {cuisine} food. Suggest a fancy name. Return only one name without any other text"
)

prompt_template_name2.format(cuisine="Mexican")

name_chain2 = LLMChain(llm=model, prompt=prompt_template_name2, output_key="restaurant_name")

prompt_template_items2 = PromptTemplate(
    input_variables= ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list"
)

food_items_chain2 = LLMChain(llm=model, prompt=prompt_template_items2, output_key="menu_items")



In [15]:
from langchain.chains.sequential import SequentialChain

chain = SequentialChain(
    chains = [name_chain2, food_items_chain2],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items' ]
)

chain.invoke({'cuisine':'arabic'})

{'cuisine': 'arabic',
 'restaurant_name': 'Sahara Nights',
 'menu_items': 'Here are some menu item suggestions for Sahara Nights:\n\nGrilled lamb chops with a spicy harissa sauce, Moroccan tagine with chicken and apricots, slow-cooked chickpeas in a rich and creamy tahini sauce, crispy falafel with a side of tangy tzatziki sauce, grilled salmon with a zesty lemon and herb marinade, hearty beef stew with a blend of aromatic spices, pan-seared shrimp with a garlic and parsley butter, Moroccan spiced sweet potato soup with crumbled feta cheese.'}

In [17]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import Tool, DuckDuckGoSearchRun, ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.llms import OpenAI

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikiTool= Tool(  
     name="Wikipedia",  
     func=wiki.run,  
     description="useful when you need an answer about encyclopedic general knowledge"  
 )

tools = [wikiTool]

agent = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("When did india got its indedence? How many years passed since independence?")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I can provide information on this topic.

Question: When did India get its independence? 
How many years passed since independence?
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 